In [1]:
import datetime

import holidays


import graphlab
import pandas as pd
import numpy as np
import pylab as plt
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
from math import sin, cos, sqrt, atan2, radians
# Set the global default size of matplotlib figures
%matplotlib inline
plt.rc('figure', figsize=(10, 5))

# Size of matplotlib figures that contain subplots
fizsize_with_subplots = (10, 10)

# Size of matplotlib histogram bins
bin_size = 10

graphlab.product_key.set_product_key('3AD3-27D5-13DA-DFDB-D8B3-4F0F-1400-873F')
graphlab.set_runtime_config('GRAPHLAB_DEFAULT_NUM_PYLAMBDA_WORKERS', 4)

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1505209329.log
INFO:graphlab.cython.cy_server:GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1505209329.log


This non-commercial license of GraphLab Create for academic use is assigned to bittu.arvind@gmail.com and will expire on August 05, 2018.


In [2]:
#df_train = pd.read_csv('data.csv')
train = graphlab.SFrame.read_csv('train_modified.csv')
#train = pd.read_csv("trained_featured_data3.csv")
test = graphlab.SFrame.read_csv('test_modified.csv')
#df_train = graphlab.SFrame.read_csv('data.csv')
#train = np.genfromtxt ('trained_featured_data2.csv', delimiter=",")

Finished parsing file /home/ubuntu/train_modified.csv

Parsing completed. Parsed 100 lines in 37.1109 secs.

Read 874505 lines. Lines per second: 25699.8

Finished parsing file /home/ubuntu/train_modified.csv

Parsing completed. Parsed 1458644 lines in 39.1062 secs.

...

Parsing completed. Parsed 100 lines in 25.1162 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,str,int,str,int,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
------------------------------------------------------

Finished parsing file /home/ubuntu/test_modified.csv

Parsing completed. Parsed 625134 lines in 14.0489 secs.


Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,str,int,str,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


In [3]:
def is_Weekend (row):
  # time data '2016-03-14 17:24:55' does not match format '%y-%m-%d %H:%M:%S'
   val = 0
   pickup_datetime = row['pickup_datetime'] #2016-03-14 17:24:55 6/30/2016 23:59
#datetime_object = datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p')
   pickup_datetime2 = datetime.datetime.strptime(pickup_datetime, "%Y-%m-%d %H:%M:%S")
   weekno = pickup_datetime2.weekday()
   if weekno<5:
    val = 0 
   else:
    val = 1 
      
   return val

In [4]:
def is_PM (row):
   
   val = 0
   pickup_datetime = row['pickup_datetime'] #2016-03-14 17:24:55
#datetime_object = datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p')
   pickup_datetime2 = datetime.datetime.strptime(pickup_datetime, "%Y-%m-%d %H:%M:%S")
   hour = pickup_datetime2.hour
   if hour > 12 :
    val = 1
      
   return val

In [5]:
def is_holiday (row):
   us_holidays = holidays.UnitedStates()
   val = 0
   pickup_datetime = row['pickup_datetime'] #2016-03-14 17:24:55
#datetime_object = datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p')
   pickup_datetime2 = datetime.datetime.strptime(pickup_datetime, "%Y-%m-%d %H:%M:%S")
   only_date = pickup_datetime2.date()
   if only_date in us_holidays :
    val = 1
      
   return val

In [6]:
train['is_holiday'] = train.apply (lambda row: is_holiday (row))
train['is_PM'] = train.apply (lambda row: is_PM (row))
train['is_Weekend'] = train.apply (lambda row: is_Weekend (row))

In [ ]:
train.head()

id,vendor_id,pickup_datetime,passenger_count,store_and_fwd_flag,trip_duration,distance_covered
id2875421,2,2016-03-14 17:24:55,1,N,455,0.931399986841
id2377394,1,2016-06-12 00:43:35,1,N,663,1.12220622904
id3858529,2,2016-01-19 11:35:24,1,N,2124,3.96863409252
id3504673,2,2016-04-06 19:32:31,1,N,429,0.923305989624
id2181028,2,2016-03-26 13:30:55,1,N,435,0.738762712149
id0801584,2,2016-01-30 22:01:40,6,N,443,0.683043576064
id1813257,1,2016-06-17 22:34:59,4,N,341,0.824343489926
id1324603,2,2016-05-21 07:54:58,1,N,1551,3.55212483979
id1301050,1,2016-05-27 23:12:23,1,N,255,0.814445182884
id0012891,2,2016-03-10 21:45:01,1,N,1225,3.18303881827


In [ ]:
train.export_csv("traning_data_with_Extra_Features3.csv")

In [ ]:
def rmsle(real,predicted):
    sum=0.0
    for x in range(len(predicted)):
        p = np.log(predicted[x]+1)
        r = np.log(real[x]+1)
        sum = sum + (p - r)**2
    return (sum/len(predicted))**0.

In [ ]:
#distance_model = graphlab.regression.create(train, target='trip_duration', features=['distance_covered','vendor_id','passenger_count','store_and_fwd','holiday','is_AM','is_PM','is_Weekend','is_jan','is_feb','is_mar','is_apr','is_may','is_jun','is_jul','is_aug','is_sep','is_oct','is_nov','is_dec'],validation_set=None,verbose=False)

In [ ]:
#distance_model = graphlab.regression.create(train, target='trip_duration', features=['is_PM','distance_covered','vendor_id','store_and_fwd_flag'],validation_set=None,verbose=False)

In [ ]:
#distance_covered : 5195.31689453
#'distance_covered','vendor_id': 5173.34228516
#'distance_covered','vendor_id','passenger_count': 5174.95654297
#'distance_covered','vendor_id','store_and_fwd_flag': 5172.52832031
#'is_PM','distance_covered','vendor_id','store_and_fwd_flag': 

In [ ]:
#print distance_model['training_rmse']

In [ ]:
#predictions = distance_model.predict(train)
#realValues = train['trip_duration']
#print realValues1
#print predictions
#finalResult = rmsle(realValues,predictions)
#print finalResult


In [ ]:
#predictions2 = distance_model.predict(test)
#predictions2.save("result1234.csv",'csv')

In [ ]:
#print distance_model['l1_penalty']
#coefs = distance_model['coefficients']
#print coefs

In [ ]:
#distance_model = graphlab.linear_regression.create(train, target='trip_duration', features=['distance_covered','vendor_id','passenger_count','store_and_fwd'],validation_set=None,verbose=False,solver = 'lbfgs',feature_rescaling = True,max_iterations = 2000,l2_penalty = 1.0,l1_penalty = 1.0)
#distance_model = graphlab.linear_regression.create(train, target='trip_duration', features=['distance_covered','vendor_id','passenger_count','store_and_fwd'],validation_set=None,verbose=False,solver = 'fista',feature_rescaling = True,max_iterations = 2000,l2_penalty = 1.0,l1_penalty = 1.0)
#distance_model = graphlab.linear_regression.create(train, target='trip_duration', features=['distance_covered','vendor_id','passenger_count','store_and_fwd','holiday','is_jan','is_feb','is_mar','is_apr','is_may','is_jun','is_jul','is_aug','is_sep','is_oct','is_nov','is_dec','is_AM','is_PM'],validation_set=None,verbose=False,solver = 'fista',feature_rescaling = True,max_iterations = 2000,l2_penalty = 1.0,l1_penalty = 1.0)
#distance_model = graphlab.linear_regression.create(train, target='trip_duration', features=['distance_covered','vendor_id','passenger_count','store_and_fwd','holiday','is_jan','is_feb','is_mar','is_apr','is_may','is_jun','is_jul','is_aug','is_sep','is_oct','is_nov','is_dec','is_AM','is_PM'],validation_set=None,verbose=False)
#distance_model = graphlab.linear_regression.create(train, target='trip_duration', features=['distance_covered'],validation_set=None,verbose=False)
#distance_model = graphlab.linear_regression.create(train, target='trip_duration', features=['holiday'],validation_set=None,verbose=False)

#predictions = distance_model.predict(test)
#results = distance_model.evaluate(test)
#fields = distance_model.list_fields()
#print fields

In [ ]:
#print distance_model['training_rmse']

In [ ]:
#print distance_model['l1_penalty']
#coefs = distance_model['coefficients']
#print coefs

In [ ]:
#print distance_model['l2_penalty']
#print distance_model['l1_penalty']
#print distance_model['max_iterations']

In [ ]:
#train.head()

In [ ]:
#data =  graphlab.SFrame('https://static.turi.com/datasets/regression/houses.csv')
#model = graphlab.linear_regression.create(data,
  #                                   target='price',
#                                  features=['bath', 'bedroom', 'size'])
#results = model.predict(data)